# Feature Extraction using Unsupervised Methods

This notebook augments the X_train and X_test dataframes with additional features from PCA extraction, MDS, and K-Means clustering techniques.

Additionally, it preserves all objects as python 'pickle' files so that results can be replicated in the future.

In [1]:
#123456789012345678901234567890123456789012345678901234567890123456789012345678

In [2]:
import sys
PATH_TO_MODULES = '../..'
sys.path.insert(0, PATH_TO_MODULES)

from helpers import *
from unsupervised_helpers import *

In [3]:


def extract_features(input_df: pd.DataFrame, pickle_path='pickles/') -> pd.DataFrame:
    """Extract features from a training dataset using PCA.

    Use this function only on the TRAINING set.
    This function extracts features using PCA and assigns cluster labels
    using K-Means.  It also saves an augmented dataframe as a CSV file, and 
    preserves the objects used to transform the data as 'pickle' files.
    """
    dataset = input_df.copy()
    if 'Unnamed: 0' in dataset.columns:
        dataset.drop(columns=['Unnamed: 0'], inplace=True)
    # Process for PCA
    pca_df = process_for_PCA(dataset)
    # We found that the top 50 PCs explain 85% of the variance
    pca = PCA(50)
    X_PCA = pca.fit_transform(pca_df)
    # Pickle the pca object
    with open(pickle_path + 'pca_all.pickle', 'wb') as f:
        pickle.dump(pca, f)
    # Make a dataframe of the top 50 PCs
    PCA_cols = ['PCA_all_PC' + str(i + 1) for i in range(X_PCA.shape[1])]
    top50_PC_all_df = pd.DataFrame(X_PCA, columns=PCA_cols)
    # Perform PCA using only the KPI subset
    subset_df = get_KPI(dataset)
    subset_pca_df = process_for_PCA(subset_df)
    pca_KPI = PCA(10)
    X_PCA_KPI = pca_KPI.fit_transform(pca_df)
    # pickle the pca_KPI object
    with open(pickle_path + 'pca_KPI.pickle', 'wb') as f:
        pickle.dump(pca_KPI, f)
    # Make a dataframe of the top 10 PCs from the KPI subset
    PCA_KPI_cols = ['PCA_KPI_PC' + str(i + 1) for i in range(X_PCA_KPI.shape[1])]
    top10_PC_KPI_df = pd.DataFrame(X_PCA_KPI, columns=PCA_KPI_cols)
    # Use K-Means to assign cluster labels
    kmeans = KMeans(n_clusters=7, init='random', n_init=100, copy_x=False)
    cluster = kmeans.fit_predict(X_PCA_KPI)
    # Pickle the K-Means object
    with open(pickle_path + 'kmeans.pickle', 'wb') as f:
        pickle.dump(kmeans, f)
    # Create augmented DataFrame
    augmented_df = pd.concat(
        (
            dataset,
            top50_PC_all_df,
            top10_PC_KPI_df,
            pd.Series(cluster, name='Cluster')
        ), axis = 1
    )
    return augmented_df

    
def transform_features(input_df: pd.DataFrame) -> pd.DataFrame:
    """Augment a dataframe with extracted features.

    Use this function on the test set.
    This function uses pickled objects that were used during feature extraction
    on the test set to augment the test set with additional features.
    """
    dataset = input_df.copy()
    if 'Unnamed: 0' in dataset.columns:
        dataset.drop(columns=['Unnamed: 0'], inplace=True)
    # Need to finish this....
    return dataset
    
    


## Load data


In [4]:
# Import merged dataset
path = PATH_TO_MODULES + '/datasets/'
input_filename = 'X_train_filled_KPIs_QoQ.csv'
dataset = pd.read_csv(path + input_filename)
print(f'There are {dataset.shape[0]} rows and {dataset.shape[1]} columns in the dataset.')

There are 1905 rows and 284 columns in the dataset.


In [7]:
augmented_df = extract_features(dataset, pickle_path='../../pickles/')
augmented_df

,Ticker,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,CashAndSTInvestments_2024Q4,...,PCA_KPI_PC2,PCA_KPI_PC3,PCA_KPI_PC4,PCA_KPI_PC5,PCA_KPI_PC6,PCA_KPI_PC7,PCA_KPI_PC8,PCA_KPI_PC9,PCA_KPI_PC10,Cluster
0,VRNT,VERINT SYSTEMS INC,Information Technology,-3981000.0,-7440000.0,-7660000.0,-6337000.0,1.854600e+08,2.078450e+08,182823000.0,...,3.219681,-2.605382,1.404603,-4.328504,-3.483180,-3.635891,0.909061,1.108912,-1.746444,2
1,CCK,CROWN HOLDINGS INC,Materials,-84000000.0,-76000000.0,-149000000.0,-33000000.0,1.414000e+09,1.738000e+09,918000000.0,...,-2.393730,4.916988,-2.095113,0.432395,-1.995545,3.386968,-4.609344,-2.426234,-1.657147,5
2,NWPX,NWPX INFRASTRUCTURE INC,Industrials,-6064000.0,-5975000.0,-4190000.0,-3670000.0,4.528000e+06,5.723000e+06,5007000.0,...,-0.657417,-1.908015,-3.350384,2.982486,4.972409,-0.081640,-3.554487,-0.575342,-2.487519,2
3,SSD,SIMPSON MANUFACTURING INC,Industrials,-40210000.0,-45226000.0,-55509000.0,-50165000.0,3.548510e+08,3.394270e+08,239371000.0,...,0.191461,-2.950287,-3.960399,0.364794,0.223856,4.791476,0.325587,-1.164209,-4.214194,1
4,PRMB,PRIMO BRANDS CLASS A CORP,Consumer Staples,-47300000.0,-41300000.0,-57600000.0,-69500000.0,1.550000e+07,1.748000e+08,613700000.0,...,-2.856722,-0.676636,5.000834,-1.767578,1.080197,-0.410473,5.793042,-0.568627,-1.811654,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900,HOPE,HOPE BANCORP INC,Financials,-2533000.0,-2900000.0,-2267000.0,-2659000.0,6.540440e+08,6.808570e+08,458199000.0,...,3.569894,5.031345,-2.125840,5.958538,-2.658913,0.202435,0.355600,-0.146524,0.950035,4
1901,CLOV,CLOVER HEALTH INVESTMENTS CORP CLA,Health Care,-397000.0,-399000.0,-315000.0,-185000.0,2.547710e+08,2.879560e+08,194543000.0,...,-3.390949,2.047995,-4.356079,-1.441836,0.538156,-1.530069,4.497612,-1.602234,2.424748,2
1902,FYBR,FRONTIER COMMUNICATIONS PARENT INC,Communication,-626000000.0,-699000000.0,-792000000.0,-757000000.0,1.197000e+09,1.320000e+09,750000000.0,...,-3.548954,5.196753,5.591376,-3.261909,-2.585422,-3.879498,0.824249,0.360135,-1.209454,6
1903,AAMI,ACADIAN ASSET MANAGEMENT INC,Financials,-2600000.0,-2400000.0,-2500000.0,-2500000.0,7.160000e+07,5.360000e+07,94800000.0,...,3.003135,-4.041884,0.849882,0.803610,1.514509,-4.650266,0.630260,-3.772502,1.274199,2
